In [1]:
#Título: Script Python para Análise de Cesta de Compras via Algoritmo de Regras de Associação
#Autor: Lucas Lai Barbosa

#Importação das bibliotecas necessárias
import pypyodbc as podbc
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore', '.*pandas only support SQLAlchemy.*', )

from mlxtend.frequent_patterns import fpgrowth, association_rules

In [2]:
#Conexão com o servidor
conn = podbc.connect("Driver={SQL Server};Server=192.168.99.63;Database=ODS;Trusted_Connection=yes")

In [3]:
#Ajuste de visualização do Jupyter Notebook
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 110)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [4]:
#Consulta SQL ao banco de dados para extração das informações das compras
SQL_Query = pd.read_sql_query('''WITH ITENS_1 AS (
	SELECT 
	--RIGHT(ID_LOJA, 4) AS 'ID_LOJA',
	CONCAT(DATA_COMPRA,'_' , ID_COMPRA) 'DATA_ID_COMPRA',
	CASE WHEN CONCAT(DEPARTAMENTO_PRODUTO, ' - ', SUBGRUPO_PRODUTO) = 'CINTO - HOMEM' THEN 'OUTROS ACESS - ACS CINTO MASCULINO' 
		 WHEN CONCAT(DEPARTAMENTO_PRODUTO, ' - ', SUBGRUPO_PRODUTO) IN ('MEIA - MULHER','MEIA - MEIA CALCA') THEN 'MEIA - ACS MEIA FEMININO'
		 ELSE CONCAT(DEPARTAMENTO_PRODUTO, ' - ', SUBGRUPO_PRODUTO) END 'DEP_SUBGRUPO',
	QUANTIDADE_PRODUTO
FROM
	CDP.TRANSACIONAL_VENDA
WHERE
	DATA_COMPRA BETWEEN '2022-03-01' AND '2022-05-31'
	AND OPERACAO_PRODUTO LIKE 'COMPRA%'
	AND SUBGRUPO_PRODUTO IS NOT NULL
	AND ORIGEM IN ('STOREX', 'PDV', 'WHATSAPP')

),
ITENS_2 AS (
SELECT 
	DATA_ID_COMPRA,
	COUNT(DEP_SUBGRUPO) AS 'QTD'
FROM ITENS_1
GROUP BY DATA_ID_COMPRA
HAVING COUNT(DEP_SUBGRUPO) > 1
)

SELECT * FROM 
(
	SELECT 
	I.DATA_ID_COMPRA,
	I.DEP_SUBGRUPO,
	I.QUANTIDADE_PRODUTO
	FROM ITENS_1 I 
		INNER JOIN ITENS_2 I2 ON I.DATA_ID_COMPRA = I2.DATA_ID_COMPRA
) t
PIVOT (
	SUM(QUANTIDADE_PRODUTO)
	FOR DEP_SUBGRUPO IN ([FECHADO - FEM BOTA CANO MD/AL],[MENINA - INF MNA JUV SANDALIA],[FECHADO - FEM TNS C CADARCO],[MENINO - INF MNO RECEM NASCIDO],[FECHADO - FEM SPT ALTO],[MENINO - INF MNO PP TENIS],[ABERTO - FEM SND ALTA],[CASUAL - MASC BOTA CASUAL],[ACESS FEM - ACS CINTO FEMININO],[OUTROS ACESS - ACS CHAPEU INFANTIL],[MENINA - INF MNA JUV SAPATO],[CINTO - HOMEM],[FECHADO - FEM SPT RASTEIRO],[MASCULINO - TNS MASC TREINO],[FEMININO - TNS FEM MULTIESPORTES],[MENINO - INF MNO JUV BOTA],[CASUAL - MASC SPT CADARCO],[FEMININO - CHL FEMININO],[CHECKSTAND - ACS PAPELARIA],[ACESS FEM - ACS BOLSA FEMININO],[OUTROS ACESS - ACS CHAPEU ADULTO],[FECHADO - FEM SPT MEDIO],[SOCIAL - MASC SPT SOCIAL],[MENINO - INF MNO INF BOTA],[MENINO - INF MNO INF TENIS],[MENINA - INF MNA BB TENIS],[MEIA - ACS MEIA MASCULINO],[MENINA - INF MNA JUV BOTA],[CASUAL - MASC SPT S CADARCO],[OUTROS ACESS - ACS CINTO MASCULINO],[FECHADO - FEM TNS S CADARCO],[MEIA - MULHER],[ABERTO - FEM SND BAIXA],[CALCADO - ALTO],[CASUAL - MASC SAND CASUAL],[MENINA - INF MNA JUV TENIS],[OUTROS ACESS - ACS CARTEIRA MASCULINO],[ABERTO - FEM TCO MEDIO],[MENINO - INF MNO INF SANDALIA],[MENINA - INF MNA PP SAPATO],[MENINO - INF MNO BB SANDALIA],[MENINO - INF MNO BB SAPATO],[OUTROS ACESS - ACS BOLSA INFANTIL],[MENINO - INF MNO PP SANDALIA],[MENINO - INF MNO INF SAPATO],[ABERTO - FEM SND RASTEIRA],[INFANTIL - CHL MNO INF],[MENINA - INF MNA RECEM NASCIDO],[INFANTIL - CHL MNA INF],[OUTROS ACESS - ACS MOCHILA INFANTIL],[CALCADO - CASUAL],[ACESS FEM - ACS CARTEIRA FEMININO],[ABERTO - FEM TCO RASTEIRO],[CASUAL - MASC BOTA OUTDOOR],[OUTROS ACESS - ACS MOCHILA ADULTO],[FEMININO - TNS FEM CORRIDA],[MENINO - INF MNO PP SAPATO],[MENINA - INF MNA PP SANDALIA],[INFANTIL - CHL MNA PP],[MENINA - INF MNA INF BOTA],[MASCULINO - CHL MASC PRAIA],[ - ],[MENINO - INF MNO JUV SAPATO],[ABERTO - FEM TCO ALTO],[MENINA - INF MNA INF SAPATO],[CHECKSTAND - ACS ALMOFADA],[FECHADO - FEM BOTA CANO BX],[FECHADO - FEM SAPATILHA],[MENINA - INF MNA PP TENIS],[MENINA - INF MNA PP BOTA],[MENINA - INF MNA BB SAPATO],[MASCULINO - TNS MASC MULTIESPORTES],[MEIA - ACS MEIA MENINA],[MENINA - INF MNA INF SANDALIA],[MASCULINO - CHL MASC CASUAL],[FECHADO - FEM BOTA COTURNO],[MENINO - INF MNO JUV SANDALIA],[MEIA - MEIA CALCA],[CHECKSTAND - ACS BELEZA],[FECHADO - FEM SPT BAIXO],[MENINO - INF MNO BB TENIS],[CALCADO - MEDIO],[CHECKSTAND - ACS PRODUTO PE],[ABERTO - FEM TCO BAIXO],[CALCADO - CONFORT],[CHECKSTAND - ACS COPO],[MEIA - ACS MEIA FEMININO],[CALCADO - BAIXO],[INFANTIL - CHL MNO PP],[CASUAL - MASC TNS CASUAL],[CHECKSTAND - ACS NECESSAIRE],[MENINO - INF MNO JUV TENIS],[PERFUMARIA - ACS PERFUMARIA],[MENINO - INF MNO PP BOTA],[ABERTO - FEM SND MEDIA],[MEIA - ACS MEIA MENINO],[CHECKSTAND - ACS CHECKSTAND OUTROS],[FEMININO - TNS FEM TREINO],[OUTROS ACESS - ACS PANTUFA],[MENINA - INF MNA INF TENIS],[MASCULINO - TNS MASC CORRIDA],[MENINA - INF MNA BB SANDALIA],[CASUAL - MASC SAPATENIS])
	) AS PIVOT_TABLE
ORDER BY 1''', conn)
data = pd.DataFrame(SQL_Query)

In [5]:
data.head(5)

,data_id_compra,fechado - fem bota cano md/al,menina - inf mna juv sandalia,fechado - fem tns c cadarco,menino - inf mno recem nascido,fechado - fem spt alto,menino - inf mno pp tenis,aberto - fem snd alta,casual - masc bota casual,acess fem - acs cinto feminino,outros acess - acs chapeu infantil,menina - inf mna juv sapato,cinto - homem,fechado - fem spt rasteiro,masculino - tns masc treino,feminino - tns fem multiesportes,menino - inf mno juv bota,casual - masc spt cadarco,feminino - chl feminino,checkstand - acs papelaria,acess fem - acs bolsa feminino,outros acess - acs chapeu adulto,fechado - fem spt medio,social - masc spt social,menino - inf mno inf bota,menino - inf mno inf tenis,menina - inf mna bb tenis,meia - acs meia masculino,menina - inf mna juv bota,casual - masc spt s cadarco,outros acess - acs cinto masculino,fechado - fem tns s cadarco,meia - mulher,aberto - fem snd baixa,calcado - alto,casual - masc sand casual,menina - inf mna juv tenis,outros acess - acs carteira masculino,aberto - fem tco medio,menino - inf mno inf sandalia,menina - inf mna pp sapato,menino - inf mno bb sandalia,menino - inf mno bb sapato,outros acess - acs bolsa infantil,menino - inf mno pp sandalia,menino - inf mno inf sapato,aberto - fem snd rasteira,infantil - chl mno inf,menina - inf mna recem nascido,infantil - chl mna inf,outros acess - acs mochila infantil,calcado - casual,acess fem - acs carteira feminino,aberto - fem tco rasteiro,casual - masc bota outdoor,outros acess - acs mochila adulto,feminino - tns fem corrida,menino - inf mno pp sapato,menina - inf mna pp sandalia,infantil - chl mna pp,menina - inf mna inf bota,masculino - chl masc praia,-,menino - inf mno juv sapato,aberto - fem tco alto,menina - inf mna inf sapato,checkstand - acs almofada,fechado - fem bota cano bx,fechado - fem sapatilha,menina - inf mna pp tenis,menina - inf mna pp bota,menina - inf mna bb sapato,masculino - tns masc multiesportes,meia - acs meia menina,menina - inf mna inf sandalia,masculino - chl masc casual,fechado - fem bota coturno,menino - inf mno juv sandalia,meia - meia calca,checkstand - acs beleza,fechado - fem spt baixo,menino - inf mno bb tenis,calcado - medio,checkstand - acs produto pe,aberto - fem tco baixo,calcado - confort,checkstand - acs copo,meia - acs meia feminino,calcado - baixo,infantil - chl mno pp,casual - masc tns casual,checkstand - acs necessaire,menino - inf mno juv tenis,perfumaria - acs perfumaria,menino - inf mno pp bota,aberto - fem snd media,meia - acs meia menino,checkstand - acs checkstand outros,feminino - tns fem treino,outros acess - acs pantufa,menina - inf mna inf tenis,masculino - tns masc corrida,menina - inf mna bb sandalia,casual - masc sapatenis
0,Apr 1 2022 1:00PM_20220401|1068|18|42303|59,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apr 1 2022 1:00PM_20220401|1076|6|23771|50,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apr 1 2022 1:00PM_20220401|1100|1|31534|11,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,N

In [6]:
data.shape

(788471, 104)

In [7]:
#Substituição dos número de peças vendidas por valores boolianos (0 ou 1)
data2 = data.fillna(0)
column_names = data2.select_dtypes(include=[np.number]).columns
data2[column_names] = data2[column_names].astype(bool)

data2.shape

(788471, 104)

In [8]:
#Descarte da coluna de identificação da compra
data2.drop(columns='data_id_compra', inplace=True)

In [9]:
#Geração das combinações de produtos mais comuns
frequent_itemsets = fpgrowth(data2, min_support=0.002, use_colnames=True)

print(frequent_itemsets.sort_values(by='support', ascending=False))

     support                                           itemsets
40      0.15                        (aberto - fem tco rasteiro)
8       0.11                        (meia - acs meia masculino)
3       0.10                      (fechado - fem tns c cadarco)
7       0.10                          (fechado - fem sapatilha)
32      0.10                         (meia - acs meia feminino)
..       ...                                                ...
211     0.00  (feminino - chl feminino, fechado - fem spt ra...
146     0.00  (feminino - chl feminino, feminino - tns fem t...
178     0.00  (fechado - fem spt medio, fechado - fem tns s ...
139     0.00  (fechado - fem sapatilha, fechado - fem bota c...
269     0.00   (infantil - chl mno inf, meia - acs meia menino)

[270 rows x 2 columns]


In [10]:
#Cálculo dos indicadores das regras de associação mais frequentes
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
rules[ (rules['confidence'] >= 0.1) &
       (rules['lift'] >= 1)].sort_values(by='confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(aberto - fem snd rasteira),(aberto - fem tco rasteiro),0.08,0.15,0.02,0.27,1.79,0.01,1.16
39,(masculino - tns masc treino),(meia - acs meia masculino),0.05,0.11,0.01,0.22,1.97,0.01,1.14
35,(aberto - fem tco baixo),(aberto - fem tco rasteiro),0.04,0.15,0.01,0.22,1.46,0.00,1.09
73,(casual - masc bota casual),(meia - acs meia masculino),0.01,0.11,0.00,0.22,1.95,0.00,1.14
78,(social - masc spt social),(meia - acs meia masculino),0.02,0.11,0.00,0.21,1.87,0.00,1.12
13,(aberto - fem tco medio),(aberto - fem tco rasteiro),0.03,0.15,0.01,0.20,1.36,0.00,1.07
53,(casual - masc tns casual),(meia - acs meia masculino),0.04,0.11,0.01,0.20,1.82,0.00,1.11
77,(social - masc spt social),(outros acess - acs cinto masculino),0.02,0.03,0.00,0.19,7.27,0.00,1.21
11,(casual - masc sapatenis),(meia - acs meia masculino),0.05,0.11,0.01,0.18,1.63,0.00,1.09
66,(acess fem - acs carteira feminino),(acess fem - acs bolsa feminino),0.01,0.03,0.00,0.18,5.16,0.00,1.18


𝑠𝑢𝑝 (𝐴) = 𝑛ú𝑚𝑒𝑟𝑜 𝑑𝑒 𝑡𝑟𝑎𝑛𝑠𝑎çõ𝑒𝑠 𝑐𝑜𝑚 𝐴 / 𝑛ú𝑚𝑒𝑟𝑜 𝑡𝑜𝑡𝑎𝑙 𝑑𝑒 𝑡𝑟𝑎𝑛𝑠𝑎çõ𝑒𝑠

𝑠𝑢𝑝 (𝐴 → 𝐵) = 𝑛ú𝑚𝑒𝑟𝑜 𝑑𝑒 𝑡𝑟𝑎𝑛𝑠𝑎çõ𝑒𝑠 𝑐𝑜𝑚 𝐴 𝑒 𝐵 / 𝑛ú𝑚𝑒𝑟𝑜 𝑡𝑜𝑡𝑎𝑙 𝑑𝑒 𝑡𝑟𝑎𝑛𝑠𝑎çõ𝑒𝑠

𝑐𝑜𝑛𝑓 (𝐴 → 𝐵) = 𝑛ú𝑚𝑒𝑟𝑜 𝑑𝑒 𝑡𝑟𝑎𝑛𝑠𝑎çõ𝑒𝑠 𝑞𝑢𝑒 𝑠𝑢𝑝𝑜𝑟𝑡𝑎𝑚 (𝐴 ∪ 𝐵) / 𝑛ú𝑚𝑒𝑟𝑜 𝑑𝑒 𝑡𝑟𝑎𝑛𝑠𝑎çõ𝑒𝑠 𝑞𝑢𝑒 𝑠𝑢𝑝𝑜𝑟𝑡𝑎𝑚 A

𝑙𝑖𝑓𝑡 (𝐴 → 𝐵) = 𝑠𝑢𝑝(𝐴 → 𝐵) / ( 𝑠𝑢𝑝 𝐴 × 𝑠𝑢𝑝(𝐵) ) 

In [ ]:
#Exportação dos resultados para planilha do Excel
rules.to_excel("Regras.xlsx")  